## Import Tools 

In [ ]:

import os
from PIL import Image
import pandas as pd
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import make_grid
from torchvision import datasets, transforms, utils
from sklearn.model_selection import train_test_split
import torchvision.models as models



### Define CNN for classification

In [ ]:

#Define CNN
learning_rate = 1e-4
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()

    
    #Define convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, padding = 2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding = 1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding = 1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5, padding=1)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=5, padding=1)

    #Define pooling layers
        self.max_pool = nn.MaxPool2d(kernel_size=5, stride = 2)


    #Define fully connected layers
        self.fc1 = nn.Linear(512*2*2,512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256,128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 1)

    #Dropout some neurons to prevent overfitting.
        self.dropout = nn.Dropout(0.5)

    #Define activation functions
        self.relu = nn.ReLU()
        self.identity = nn.Identity()


#Apply convolutional layers with pooling in between
    def forward(self, x):
        feature_map = []
        x = self.max_pool(nn.functional.relu(self.conv1(x)))
        feature_map.append(x)
        x = self.max_pool(nn.functional.relu(self.conv2(x)))
        feature_map.append(x)
        x = self.max_pool(nn.functional.relu(self.conv3(x)))
        feature_map.append(x)
        x = self.max_pool(nn.functional.relu(self.conv4(x)))
        feature_map.append(x)
        x = self.max_pool(nn.functional.relu(self.conv5(x)))
        feature_map.append(x)
        
        

#Flatten output
        x = x.view(-1, 512*2*2)

        x = self.dropout(nn.functional.relu(self.fc1(x)))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu((self.fc4(x)))
        x = self.identity(self.fc5(x))
        

        return x

model = Model()
device = torch.device('cpu')  # use cuda or cpu
model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print(model)

### Load data 

In [ ]:
data_path = "C:/Users/sebas/OneDrive/Dokumenter/skole/4 Semester/Fagprojekt/Data/RedditDataWithLinks.csv"
posts_tidy_df = pd.read_csv(data_path)
pd.set_option('display.max_columns', None)
posts_tidy_df

In [ ]:

def ImageWithScore(img_path, csv):
    data_path = pd.read_csv(csv)
    transformed_images = []
    scores = []
    corrupted = []
    counter = 0

    for index, row in data_path.iterrows():
        if counter % 1000 == 0:
            print('Current progress is at: {count}'.format(count=counter))

        submission_id = row['PostID'] + ".jpg"
        score = np.log10(row['AppliedScale']) #Choose target here!!
        image_path = os.path.join(img_path, submission_id)
        image_path = os.path.join(image_path).replace("\\", "/")
        if os.path.exists(image_path):
            try:
                with Image.open(image_path) as image:
                    # Normalize to mean and std of ImageNet
                    transform = transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                    ])

                    transformed_image = transform(image)
                    transformed_images.append(transformed_image)
                    scores.append(score)
                    counter += 1
            except Exception as e:
                print(f'Image failed: {submission_id}: {e}')
                corrupted.append(submission_id)
                counter += 1
        else:
            print(f'Image not found: {image_path}')
            counter += 1

    return transformed_images, scores

image_path = "C:/Users/sebas/OneDrive/Dokumenter/skole/4 Semester/Fagprojekt/Images/"
data_path = "C:/Users/sebas/OneDrive/Dokumenter/skole/4 Semester/Fagprojekt/Data/RedditDataWithLinks.csv"

transformed_images, scores = ImageWithScore(image_path, data_path)


#### Split into test/train

In [ ]:

train_images, test_images, train_scores, test_scores = train_test_split(transformed_images, scores, test_size=0.2, random_state=42)

train_data = list(zip(train_images, train_scores))
test_data = list(zip(test_images, test_scores))


In [ ]:
train_loader = torch.utils.data.DataLoader(Loader(train_data), batch_size=10, shuffle=True)
test_loader = torch.utils.data.DataLoader(Loader(test_data), batch_size=10, shuffle=False)

#### Load data properly for pytorch

In [ ]:


class Loader(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image, score = self.data[index]
        return image, score





### Train CNN 

In [ ]:
# Train loop
train_loader = torch.utils.data.DataLoader(Loader(train_data), batch_size=10, shuffle=True)
test_loader = torch.utils.data.DataLoader(Loader(test_data), batch_size=10, shuffle=False)

def CNN(learning_rate, batch_size, num_epochs):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    model.train()
    train_losses = []
    test_losses = []

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        for i, (images, score) in enumerate(train_loader):
            optimizer.zero_grad()

            if images.dim() == 3:
                images = torch.unsqueeze(images, dim=0)

            output = model(images.float())
            loss = criterion(output.float(), score.float()) 
            loss = loss.mean()
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Test the model
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for data, target in test_loader:
                if data.dim() == 3:
                    data = torch.unsqueeze(data, dim=0)
                output = model(data)
                test_loss += criterion(output, target)

            avg_test_loss = test_loss / len(test_loader)
            test_losses.append(avg_test_loss)

        print('Epoch [{}/{}], Train Loss: {:.4f}, Test Loss: {:.4f}'.format(epoch + 1, num_epochs, avg_train_loss, avg_test_loss))

    print("Finished training.")

    return model, train_losses, test_losses

model, train_losses, test_losses = CNN(learning_rate=1e-4, batch_size=10, num_epochs=15)


### Save model 

In [ ]:
torch.save(model.state_dict(), 'C:/Users/sebas/OneDrive/Dokumenter/skole/4 Semester/Fagprojekt/Data/saved_model.pth')

In [ ]:
## Mean score
train_scores = []
for images, scores in train_loader:
    train_scores.extend(scores.tolist())

mean_score = sum(train_scores) / len(train_scores)
print("Mean of train set scores:", mean_score)


In [ ]:
## Median score
train_scores = []
for images, scores in train_loader:
    train_scores.extend(scores.tolist())

median_score = np.median(train_scores)
print("Median of train set scores:", median_score)


#### Predict scores (Our own)

In [ ]:
# Load model and change last layer and evaluate
model = Model()

model.load_state_dict(torch.load('C:/Users/sebas/OneDrive/Dokumenter/skole/4 Semester/Fagprojekt/Data/saved_model.pth'))
model.to(device)
model.eval()

predicted_scores = []
originals = []
predictions = []

with torch.no_grad():
    for images, scores in test_loader:
        images = images.to(device)
        outputs = model(images)
        predicted_scores.extend(outputs.squeeze().cpu().numpy())

        # Make tuples for sorting easily
        original_tuples = list(zip(images, scores))
        originals.extend(original_tuples)
        predicted_tuples = list(zip(images, outputs.squeeze().cpu().numpy()))
        predictions.extend(predicted_tuples)


### Plot scores

In [ ]:


# Convert the predicted_scores list to a numpy array
predicted_scores = np.array(predicted_scores)

# Convert the target scores to a numpy array
actual_scores = np.array(test_scores)

# Plotting the predicted scores
plt.figure(figsize=(10, 6))
plt.scatter(range(len(predicted_scores)), predicted_scores, label='Predicted Scores')
plt.xlabel('Image Index')
plt.ylabel('Predicted Score')
plt.title('Predicted Scores')
plt.legend()
plt.show()

# Plotting the actual scores
plt.figure(figsize=(10, 6))
plt.scatter(range(len(actual_scores)), actual_scores, label='Actual Scores')
plt.xlabel('Image Index')
plt.ylabel('Actual Score')
plt.title('Actual Scores')
plt.legend()
plt.show()

#### Images with highest and lowest score

In [ ]:
#Inverse ImageNet normalisation
normalise_inverse = transforms.Normalize(
    mean=[-0.485 / 0.229, -0.456 / 0.224, -0.406 / 0.225],
    std=[1 / 0.229, 1 / 0.224, 1 / 0.225]
)

sorted_original_data = sorted(originals, key=lambda x: x[1])
sorted_predicted_data = sorted(predictions, key=lambda x: x[1])

top_10_high_original = sorted_original_data[-10:]
top_10_low_original = sorted_original_data[:10]

top_10_high_predicted = sorted_predicted_data[-10:]
top_10_low_predicted = sorted_predicted_data[:10]

def plot_images_with_scores(images, scores, title):
    plt.figure(figsize=(12, 6))
    plt.suptitle(title, fontsize=16)
    for i, (image, score) in enumerate(zip(images, scores)):
        denormalized_image = normalise_inverse(image)
        plt.subplot(2, 5, i + 1)
        plt.imshow(denormalized_image.permute(1, 2, 0).cpu().numpy())
        plt.title(f"Score: {score.item():.2f}") 
        plt.axis('off')
    plt.tight_layout()
    plt.show()

predicted_scores = [score.item() for _, score in predictions]


plot_images_with_scores([image for image, _ in top_10_high_original], [score for _, score in top_10_high_original], "Top 10 Images with Highest Original Scores")
plot_images_with_scores([image for image, _ in top_10_low_original], [score for _, score in top_10_low_original], "Top 10 Images with Lowest Original Scores")
plot_images_with_scores([image for image, _ in top_10_high_predicted], [score for _, score in top_10_high_predicted], "Top 10 Images with Highest Predicted Scores")
plot_images_with_scores([image for image, _ in top_10_low_predicted], [score for _, score in top_10_low_predicted], "Top 10 Images with Lowest Predicted Scores")


#### Test model on actual top and bottom 10

In [ ]:
test_images = [image for image, _ in top_10_high_original]

predicted_scores_top_10 = []

with torch.no_grad():
    for image in test_images:
        image = image.to(device)
        output = model(image.unsqueeze(0))
        predicted_scores_top_10.append(output.item())

print("Top 10 Actual and Predicted Scores:")
for i, (image, actual_score) in enumerate(top_10_high_original):
    predicted_score = predicted_scores_top_10[i]
    print(f"Image {i+1}: Actual Score = {actual_score:.2f}, Predicted Score = {predicted_score:.2f}")

In [ ]:
test_images = [image for image, _ in top_10_low_original]

predicted_scores_bottom_10 = []

with torch.no_grad():
    for image in test_images:
        image = image.to(device)
        output = model(image.unsqueeze(0))
        predicted_scores_bottom_10.append(output.item())

print("Bottom 10 Actual and Predicted Scores:")
for i, (image, actual_score) in enumerate(top_10_low_original):
    predicted_score = predicted_scores_bottom_10[i]
    print(f"Image {i+1}: Actual Score = {actual_score:.2f}, Predicted Score = {predicted_score:.2f}")


In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(actual_scores_top_10, predicted_scores_top_10, label='Top 10')
plt.scatter(actual_scores_bottom_10, predicted_scores_bottom_10, label='Bottom 10')
plt.xlabel('Actual Score')
plt.ylabel('Predicted Score')
plt.title('Actual Scores vs Predicted Scores')
plt.xlim(y_min, y_max)
plt.ylim(-1, y_max)
plt.plot([y_min, y_max], [y_min, y_max], color='red', linestyle='--')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
min_score = min(np.min(predicted_scores), np.min(actual_scores))
max_score = max(np.max(predicted_scores), np.max(actual_scores))
y_min, y_max = min_score * 1.1, max_score * 1.1 

predicted_scores = np.array(predicted_scores)
actual_scores = np.array(actual_scores)

### Plotting actual scores and predicted scores

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(actual_scores, predicted_scores)
plt.xlabel('Actual Score')
plt.ylabel('Predicted Score')
plt.title('Actual Scores vs Predicted Scores')
plt.xlim(y_min, y_max)
plt.ylim(y_min, y_max)
plt.plot([y_min, y_max], [y_min, y_max], color='red', linestyle='--')
plt.grid(True)
plt.show()

### Visualize feature map 

#### Extract features (Lowest upvoted)

In [ ]:
model = Model()

model.load_state_dict(torch.load('C:/Users/sebas/OneDrive/Dokumenter/skole/4 Semester/Fagprojekt/Data/saved_model.pth'))
model.to(device)

lowest_score_image = top_10_low_predicted[0][0]
input_tensor = lowest_score_image.unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    # Visualize activation map for conv1
    conv1_features = model.conv1(input_tensor)
    conv1_activation_maps = conv1_features.squeeze(0).cpu().numpy()

# Visualize the activation maps
fig, axes = plt.subplots(nrows=1, ncols=8, figsize=(12, 2))

# Visualize conv1 activation maps
for i, ax in enumerate(axes):
    ax.imshow(conv1_activation_maps[i])
    ax.axis('off')
axes[0].set_title('conv1')

plt.tight_layout()
plt.show()

#### Extract features (Highest upvoted)

In [ ]:
model = Model()

model.load_state_dict(torch.load('C:/Users/sebas/OneDrive/Dokumenter/skole/4 Semester/Fagprojekt/Data/saved_model.pth'))
model.to(device)

highest_score_image = top_10_high_predicted[-1][0]
input_tensor = highest_score_image.unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    # Visualize activation map for conv1
    conv1_features = model.conv1(input_tensor)
    conv1_activation_maps = conv1_features.squeeze(0).cpu().numpy()

# Visualize the activation maps
fig, axes = plt.subplots(nrows=1, ncols=8, figsize=(12, 2))

# Visualize conv1 activation maps
for i, ax in enumerate(axes):
    ax.imshow(conv1_activation_maps[i])
    ax.axis('off')
axes[0].set_title('conv1')

plt.tight_layout()
plt.show()

#### Saliency Map (Highest upvote)

In [ ]:
model = Model()

model.load_state_dict(torch.load('C:/Users/sebas/OneDrive/Dokumenter/skole/4 Semester/Fagprojekt/Data/saved_model.pth'))
model.to(device)
model.eval()

# Define the input tensor
input_tensor = torch.tensor(highest_score_image, device=device, dtype=torch.float32)
input_tensor = normalise_inverse(input_tensor)
input_tensor = input_tensor.unsqueeze(0).requires_grad_()

output = model(input_tensor)

model.zero_grad()
output.backward()

gradients = input_tensor.grad[0].detach().cpu()

grayscale_gradients = np.abs(gradients.detach().cpu().numpy()).mean(axis=0)

normalized_gradients = (grayscale_gradients - np.min(grayscale_gradients)) / (
    np.max(grayscale_gradients) - np.min(grayscale_gradients)
)

# Plot the input image and the saliency map
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(input_tensor.squeeze().permute(1, 2, 0).detach().cpu().numpy())
axes[0].axis('off')
axes[0].set_title('Input Image')
axes[1].imshow(normalized_gradients, cmap='hot')
axes[1].axis('off')
axes[1].set_title('Saliency Map')
plt.tight_layout()
plt.show()

#### Saliency Map (Lowest upvoted)

In [ ]:


# Define the model and load the weights
model = Model()

model.load_state_dict(torch.load('C:/Users/sebas/OneDrive/Dokumenter/skole/4 Semester/Fagprojekt/Data/saved_model.pth'))
model.to(device)
model.eval()

# Define the input tensor
input_tensor = torch.tensor(lowest_score_image, device=device, dtype=torch.float32)

# Reverse normalization
normalise_inverse = transforms.Normalize(
    mean=[-0.485 / 0.229, -0.456 / 0.224, -0.406 / 0.225],
    std=[1 / 0.229, 1 / 0.224, 1 / 0.225]
)
input_tensor = normalise_inverse(input_tensor)

input_tensor = input_tensor.unsqueeze(0).requires_grad_()  # Create a new tensor with requires_grad=True

output = model(input_tensor)

model.zero_grad()
output.backward()

gradients = input_tensor.grad[0].detach().cpu()

grayscale_gradients = np.abs(gradients.detach().cpu().numpy()).mean(axis=0)

normalized_gradients = (grayscale_gradients - np.min(grayscale_gradients)) / (
    np.max(grayscale_gradients) - np.min(grayscale_gradients)
)

# Plot the input image and the saliency map
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(input_tensor.squeeze().permute(1, 2, 0).detach().cpu().numpy())
axes[0].axis('off')
axes[0].set_title('Input Image')
axes[1].imshow(normalized_gradients, cmap='hot')
axes[1].axis('off')
axes[1].set_title('Saliency Map')
plt.tight_layout()
plt.show()

### Saliency map specific picture

In [ ]:


# Assuming you have the trained model saved as 'model.pth'
model_path = 'C:/Users/sebas/OneDrive/Dokumenter/skole/4 Semester/Fagprojekt/Data/saved_model.pth'

# Load the trained model
model = Model()
model.load_state_dict(torch.load(model_path))
model.eval()

# Preprocess the input image
input_image = Image.open("{}/{}".format(image_path, "h7rkz.jpg"))
input_image = input_image.resize((224, 224))
input_tensor = transform(input_image).unsqueeze(0)

# Set the model to evaluation mode and disable gradients
model.eval()
input_tensor.requires_grad_()

# Forward pass to obtain the output
output = model(input_tensor)

# Calculate the gradients of the output with respect to the input
output.backward()

# Get the gradients from the input tensor
gradients = input_tensor.grad[0]

# Convert the gradients to grayscale
grayscale_gradients = np.abs(gradients.numpy()).mean(axis=0)

# Normalize the gradients
normalized_gradients = (grayscale_gradients - np.min(grayscale_gradients)) / (
    np.max(grayscale_gradients) - np.min(grayscale_gradients)
)

# Plot the original image and the saliency map
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(input_image)
axes[0].axis('off')
axes[0].set_title('Input Image')
axes[1].imshow(normalized_gradients, cmap='hot')
axes[1].axis('off')
axes[1].set_title('Saliency Map')
plt.tight_layout()
plt.show()
